In [514]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
spark = (SparkSession.builder.appName("project")\
.config("hive.metastore.uris","thrift://ip-10-1-2-24.ap-south-1.compute.internal:9083")\
.enableHiveSupport().getOrCreate())

In [515]:
spark

In [516]:
spark.sql('use suvamalabs').show()

++
||
++
++



In [517]:
employees=spark.sql("select * from employeesorg")
departments=spark.sql("select * from departments")
dept_managers=spark.sql("select * from dept_manager")
dept_emp=spark.sql("select * from dept_emp")
salaries=spark.sql("select * from salaries")
titles=spark.sql("select * from titles")

In [518]:
#cleaning name within quotes

departments = departments.withColumn('dept_name',regexp_replace('dept_name', '"', ''))

In [519]:
#create pandas dataframe
employees_df = employees.toPandas()
departments_df = departments.toPandas()
dept_managers_df = dept_managers.toPandas()
dept_emp_df = dept_emp.toPandas()
salaries_df = salaries.toPandas()
titles_df = titles.toPandas()

In [520]:
import pandas as pd

In [521]:
emp_data = employees_df.merge(dept_emp_df, how='left', on='emp_no')

In [522]:
emp_data_df1 = emp_data.merge(departments_df, how='left', on='dept_no')

In [523]:
emp_data_df2 = emp_data_df1.merge(salaries_df, how='left', on='emp_no')

In [524]:
emp_data_df3 = emp_data_df2.merge(titles_df, how='left', left_on = 'emp_title', right_on = 'title_id')

In [525]:
emp_data_df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 331603 entries, 0 to 331602
Data columns (total 16 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   emp_no                   331603 non-null  int32 
 1   emp_title                331603 non-null  object
 2   birth_date               331603 non-null  object
 3   first_name               331603 non-null  object
 4   last_name                331603 non-null  object
 5   sex                      331603 non-null  object
 6   hire_date                331603 non-null  object
 7   no_of_projects           331603 non-null  int32 
 8   last_performance_rating  331603 non-null  object
 9   last_date                331603 non-null  object
 10  left_org                 331603 non-null  object
 11  dept_no                  331603 non-null  object
 12  dept_name                331603 non-null  object
 13  salary                   331603 non-null  int32 
 14  title_id            

In [526]:
emp_data_df3['last_date'] = emp_data_df3['last_date'].replace(to_replace = '\r', value = '', regex = True)

In [527]:
emp_data_df3['last_performance_rating'] = emp_data_df3['last_performance_rating'].replace(to_replace = 'S',value = 4)
emp_data_df3['last_performance_rating'] = emp_data_df3['last_performance_rating'].replace(to_replace = 'A',value = 3)
emp_data_df3['last_performance_rating'] = emp_data_df3['last_performance_rating'].replace(to_replace = 'B',value = 2)
emp_data_df3['last_performance_rating'] = emp_data_df3['last_performance_rating'].replace(to_replace = 'C',value = 1)
emp_data_df3['last_performance_rating'] = emp_data_df3['last_performance_rating'].replace(to_replace = 'PIP',value = 0)

In [528]:
emp_data_df3.head(10)

,emp_no,emp_title,birth_date,first_name,last_name,sex,hire_date,no_of_projects,last_performance_rating,last_date,left_org,dept_no,dept_name,salary,title_id,title
0,473302,s0001,7/25/1953,Hideyuki,Zallocco,M,4/28/1990,2,3,,0,d002,Finance,40000,s0001,Staff
1,475053,e0002,11/18/1954,Byong,Delgrande,F,9/7/1991,1,1,,0,d004,Production,53422,e0002,Engineer
2,57444,e0002,1/30/1958,Berry,Babb,F,3/21/1992,9,3,,0,d004,Production,48973,e0002,Engineer
3,421786,s0001,9/28/1957,Xiong,Verhoeff,M,11/26/1987,2,1,,0,d003,Human Resources,40000,s0001,Staff
4,282238,e0003,10/28/1952,Abdelkader,Baumann,F,1/18/1991,6,2,,0,d006,Quality Management,40000,e0003,Senior Engineer
5,263976,e0003,10/30/1959,Eran,Cusworth,M,11/14/1986,8,2,8/21/1993,1,d006,Quality Management,40000,e0003,Senior Engineer
6,273487,s0001,4/14/1957,Christoph,Parfitt,M,6/28/1991,10,3,,0,d003,Human Resources,56087,s0001,Staff
7,461591,s0002,11/17/1964,Xudong,Samarati,M,11/13/1985,2,3,8/21/1994,1,d002,Finance,40000,s0002,Senior Staff
8,477657,e0002,12/18/1962,Lihong,Magliocco,M,10/23/1993,5,3,,0,d006,Quality Management,54816,e0002,Engineer
9,219881,s0002,4/24/1956,Kwangyoen,Speek,F,2/14/1993,6,2,,0,d009,Customer Service,40000,s0002,Senior Staff


In [529]:
emp_data_df3.isna().sum()

emp_no                     0
emp_title                  0
birth_date                 0
first_name                 0
last_name                  0
sex                        0
hire_date                  0
no_of_projects             0
last_performance_rating    0
last_date                  0
left_org                   0
dept_no                    0
dept_name                  0
salary                     0
title_id                   0
title                      0
dtype: int64

In [530]:
import matplotlib.pyplot as plt
import seaborn as sn
%matplotlib inline

In [531]:
import pandas as pd

In [532]:
emp_data_df3.columns

Index(['emp_no', 'emp_title', 'birth_date', 'first_name', 'last_name', 'sex',
       'hire_date', 'no_of_projects', 'last_performance_rating', 'last_date',
       'left_org', 'dept_no', 'dept_name', 'salary', 'title_id', 'title'],
      dtype='object')

In [533]:
#create spark dataframe
emp_data_df4 = spark.createDataFrame(emp_data_df3)

In [534]:
from pyspark.sql.functions import lit,col

In [535]:
#Columns that will be used as features and their types

continuous_features = ['salary','no_of_projects']
                    
categorical_features = ['dept_no','title_id', 'sex']

In [536]:
#Encoding all categorical features

from pyspark.ml.feature \
import OneHotEncoderEstimator, StringIndexer, VectorAssembler, PolynomialExpansion, VectorIndexer

In [537]:
#Create object of StringIndexer class and specify input and output column

SI_dept_no = StringIndexer(inputCol='dept_no',outputCol='dept_no_Idx5')
SI_sex = StringIndexer(inputCol='sex',outputCol='sex_Idx5')
SI_title_id = StringIndexer(inputCol='title_id',outputCol='title_id_Idx5')

In [538]:
#Transform the data
emp_data_df4 = SI_dept_no.fit(emp_data_df4).transform(emp_data_df4)
emp_data_df4 = SI_sex.fit(emp_data_df4).transform(emp_data_df4)
emp_data_df4 = SI_title_id.fit(emp_data_df4).transform(emp_data_df4)


In [539]:
# view the transformed data
emp_data_df4.select('dept_no', 'dept_no_Idx5', 'sex', 'sex_Idx5', 'title_id','title_id_Idx5').show(5)

+-------+------------+---+--------+--------+-------------+
|dept_no|dept_no_Idx5|sex|sex_Idx5|title_id|title_id_Idx5|
+-------+------------+---+--------+--------+-------------+
|   d002|         8.0|  M|     0.0|   s0001|          0.0|
|   d004|         1.0|  F|     1.0|   e0002|          2.0|
|   d004|         1.0|  F|     1.0|   e0002|          2.0|
|   d003|         7.0|  M|     0.0|   s0001|          0.0|
|   d006|         6.0|  F|     1.0|   e0003|          1.0|
+-------+------------+---+--------+--------+-------------+
only showing top 5 rows



In [540]:
featureCols = continuous_features + ['dept_no_Idx5','sex_Idx5','title_id_Idx5'] 

In [541]:
assembler = VectorAssembler( inputCols = featureCols, outputCol = "features")

In [542]:
train_df1 = assembler.transform(emp_data_df4 )

In [543]:
train_df1 = train_df1.withColumn('label', train_df1['left_org'].cast('integer'))

In [544]:
#Split the dataset

train_df, test_df = train_df1.randomSplit( [0.7, 0.3], seed = 42 )

### Logistic Regression

In [545]:
from pyspark.ml.classification import LogisticRegression

In [546]:
#lr = LogisticRegression(maxIter=500, regParam=0.0)

lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=10)

In [547]:
lm = lr.fit(train_df)

In [548]:
#Make predictions on train data and evaluate

y_pred_train = lm.transform(train_df)

In [549]:
y_pred_test = lm.transform( test_df )

In [550]:
y_pred_test.select( 'features',  'label', 'prediction', 'left_org' ).show(10)

+--------------------+-----+----------+--------+
|            features|label|prediction|left_org|
+--------------------+-----+----------+--------+
|[49324.0,6.0,0.0,...|    0|       0.0|       0|
|[67854.0,4.0,2.0,...|    0|       0.0|       0|
|[40000.0,5.0,0.0,...|    0|       0.0|       0|
|[40000.0,9.0,6.0,...|    0|       0.0|       0|
|[43658.0,4.0,0.0,...|    0|       0.0|       0|
|[40000.0,10.0,0.0...|    0|       0.0|       0|
|[44359.0,8.0,0.0,...|    1|       0.0|       1|
|[48342.0,4.0,8.0,...|    0|       0.0|       0|
|[71593.0,3.0,2.0,...|    0|       0.0|       0|
|[40000.0,9.0,1.0,...|    0|       0.0|       0|
+--------------------+-----+----------+--------+
only showing top 10 rows



In [552]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator()
print('Test Area under ROC', evaluator.evaluate(y_pred_test))

Test Area under ROC 0.5023182208281756


### Random Forest Classifier

In [553]:
from pyspark.ml.classification import RandomForestClassifier
RF = RandomForestClassifier( featuresCol='features', labelCol='label')
RF_model = RF.fit(train_df)

In [554]:
predictions = RF_model.transform(test_df)

In [555]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [556]:
evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')
accuracy = evaluator.evaluate(predictions)
accuracy

0.9022659187325331